# ReCentAutoma

`ReCentAutoma` is an agentic module implementation in Bridgic that provides a goal-oriented, guidance-enabled long-running autonomous agent. It enables developers to build intelligent systems that can autonomously plan and execute multi-step tasks using tools, with built-in memory management and goal tracking.

## Introduction

`ReCentAutoma` serves as a specialized agentic module in Bridgic, designed to provide a **goal-oriented**, **guidance-enabled** implementation for long-running autonomous agents. It addresses the fundamental challenges of building intelligent systems that can autonomously plan, execute, and adapt over extended interactions.

### When to Use ReCentAutoma

`ReCentAutoma` is suitable when you need to develop an application with autonomous capabilities and dynamic tool usage. In many cases, developers also need:

- **Efficient memory management**: As tasks span multiple steps and tool calls, conversation history grows. ReCentAutoma's built-in *ReCENT memory managemnt algorithm* automatically compresses history when needed, preventing context explosion.
- **Natural language goal definition**: ReCentAutoma accepts `goal` as a first-class input, so you can state your objectives directly in natural language.
- **Customized task guidance**: You can provide an optional `guidance` input to further constrain or direct the agent's behavior, enabling more control across the execution.

These capabilities are well-supported in `ReCentAutoma`, making it ideal for building autonomous agents that need to:

- Dynamically make decisions at runtime to select tools for each step
- Autonomously execute multiple steps during a long-time task
- Automatically maintain alignment with the initial goal throughout the entire execution process

### How Does ReCentAutoma Work

`ReCentAutoma` leverages `GraphAutoma`'s core features—**dynamic topology** and **dynamic routing**—to implement an enhanced **ReAct** (*Resoning and Acting*) loop. It incorporates the innovative **ReCENT** (*Recursive Compressed Episodic Node Tree*) memory management algorithm, which triggers memory compression when necessary, effectively addressing two long-standing challenges in autonomous agents:

- **Context Explosion**: As conversation history grows linearly, the context window eventually overflows. ReCENT automatically compresses episodic memory when thresholds are reached, maintaining a compact yet informative context.
- **Goal Drift**: Without explicit goal management, agents tend to deviate from original objectives during extended execution. ReCentAutoma maintains goal and guidance as persistent, non-compressible nodes, ensuring the agent stays focused on the original task.

## Quick Start

Let's explore `ReCentAutoma` through practical examples. We'll start with a simple weight management assistant using custom tools, then move to a more advanced browser automation agent using MCP integration.

First of all we have to initialize an LLM instance:

In [1]:
import dotenv
import os

from bridgic.llms.openai import OpenAILlm, OpenAIConfiguration

dotenv.load_dotenv()

_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1")
_model_name = os.environ.get("OPENAI_MODEL_NAME", "gpt-4o-mini")

# Initialize LLM (you can replace this with your own LLM instance)
llm = OpenAILlm(
    api_base=_api_base,
    api_key=_api_key,
    configuration=OpenAIConfiguration(model=_model_name),
    timeout=60,
)

### Example 1: Building a Weight Management Assistant with Custom Tools

In this example, we'll create custom tools for BMI calculation and ideal weight calculation, then use ReCentAutoma to build a weight management assistant that can help users understand their health metrics.

In [2]:
from bridgic.core.automa import RunningOptions
from bridgic.core.agentic.recent import ReCentAutoma, StopCondition

# Define custom tools for weight management
async def calculate_bmi(weight_kg: float, height_m: float) -> str:
    """
    Calculate Body Mass Index (BMI) from weight and height.

    Parameters
    ----------
    weight_kg : float
        Weight in kilograms. Must be a positive number.
    height_m : float
        Height in meters. Must be a positive number.

    Returns
    -------
    str
        A formatted string containing the BMI value and its interpretation. BMI categories:
        - Underweight: < 18.5
        - Normal: 18.5 - 24.9
        - Overweight: 25 - 29.9
        - Obese: >= 30
    """
    if weight_kg <= 0 or height_m <= 0:
        return "Error: Weight and height must be positive numbers."

    bmi = weight_kg / (height_m ** 2)

    if bmi < 18.5:
        category = "Underweight"
    elif bmi < 25:
        category = "Normal"
    elif bmi < 30:
        category = "Overweight"
    else:
        category = "Obese"

    return f"BMI: {bmi:.2f} ({category}). Weight: {weight_kg} kg, Height: {height_m} m."

# Create an agent with the custom tools (setting debug=True to see more details)
weight_assistant = ReCentAutoma(
    llm=llm,
    tools=[calculate_bmi],
    stop_condition=StopCondition(max_iteration=5, max_consecutive_no_tool_selected=1),
    running_options=RunningOptions(debug=True),
)

person_weight = 82
person_height = 1.70
person_name = "John Smith"
person_gender = "male"

result = await weight_assistant.arun(
    goal=(
        f"Calculate a person's BMI and provide personalized suggestions for effective weight management."
        f"\n- Name: {person_name}"
        f"\n- Gender: {person_gender}"
        f"\n- Weight: {person_weight} kg"
        f"\n- Height: {person_height} m"
    ),
    guidance=(
        "First calculate the BMI of the person and then give out a suggestion about the weight management."
    ),
)
print(result)


[ReCentAutoma]-[ReCentAutoma-751ecd6b] is started.
[ReCentAutoma]-[ReCentAutoma-751ecd6b] [__dynamic_step__] driving [initialize_task_goal]
[ReCentAutoma]-[ReCentAutoma-751ecd6b] [__automa__] triggers [initialize_task_goal]
[ReCentAutoma]-[ReCentAutoma-751ecd6b] 🎯 Task Goal
Calculate a person's BMI and provide personalized suggestions for effective weight management.
- Name: John Smith
- Gender: male
- Weight: 82 kg
- Height: 1.7 m

First calculate the BMI of the person and then give out a suggestion about the weight management.
[ReCentAutoma]-[ReCentAutoma-751ecd6b] [__dynamic_step__] driving [observe]
[ReCentAutoma]-[ReCentAutoma-751ecd6b] [initialize_task_goal] triggers [observe]
[ReCentAutoma]-[ReCentAutoma-751ecd6b] 👀 Observation
    Iteration: 1
    Achieved: False
    Thinking: The BMI has not yet been calculated for John Smith, and no personalized weight management suggestions have been provided. Consequently, there are significant gaps between the current state and the task g

### Example 2: Building a Browser Automation Agent with Playwright MCP Server

This example demonstrates how to integrate `ReCentAutoma` with MCP (Model Context Protocol) to create a browser automation agent. We'll use the [Playwright MCP server](https://github.com/microsoft/playwright-mcp) to enable the agent to interact with browsers like Chrome autonomously to achieve the user-defined goal.

This example demonstrates how to combine `ReCentAutoma` with MCP (Model Context Protocol) Integration to build a browser automation agent for retrieving the latest gold prices from the official website of the [Hong Kong Gold Exchange](https://hkgx.com.hk/en/).

Steps overview:

1. Connect to the Playwright MCP server via stdio transport for browser automation capabilities.
2. Register the connection using a dedicated Connection Manager to manage browser operations in isolation.
3. Create a `ReCentAutoma` agent loaded with browser automation tools.
4. Let the agent autonomously navigate to the Hong Kong Gold Exchange website and extract the needed information.

If you’re new to MCP, please check the [MCP Quick Start](../../protocol_integration/mcp_quick_start) for setup and usage instructions.

In [2]:
import os
import tempfile

from bridgic.core.automa import RunningOptions
from bridgic.core.agentic.recent import ReCentAutoma, ReCentMemoryConfig, StopCondition
from bridgic.protocols.mcp import (
    McpServerConnectionStdio,
    McpServerConnectionManager,
)

temp_dir = os.path.realpath(tempfile.mkdtemp())
print(f"✓ Using temporary directory: {temp_dir}")

# Connect to Playwright MCP server
# Note: This requires Node.js and npx to be installed
playwright_connection = McpServerConnectionStdio(
    name="connection-playwright-stdio",
    command="npx",
    args=[
        f"@playwright/mcp@latest",
        f"--output-dir={temp_dir}",
        f"--viewport-size=1920x1080",
        f"--save-video=1920x1080",
    ],
    request_timeout=60,
)

# Register the connection with a dedicated manager
McpServerConnectionManager.get_instance("browser-use").register_connection(playwright_connection)

# Establish the connection and verify connection and list available tools
playwright_connection.connect()
print(f"✓ Connected to Playwright MCP server: {playwright_connection.name}")
print(f"  Connection status: {playwright_connection.is_connected}")

# List tools
tools = playwright_connection.list_tools()
print(f"✓ Found {len(tools)} available browser tools")

# Create a browser automation agent with Playwright MCP tools
browser_agent = ReCentAutoma(
    llm=llm,
    tools=tools,
    memory_config=ReCentMemoryConfig(
        llm=llm,
        max_node_size=8,
        max_token_size=1024*32,
    ),
    stop_condition=StopCondition(max_iteration=20, max_consecutive_no_tool_selected=1),
    running_options=RunningOptions(debug=True),
)

# Use the agent to search for information on Google
search_query = "The latest price of gold"

result = await browser_agent.arun(
    goal=(
        f"Find the recent gold prices on Hong Kong Gold Exchange website."
    ),
    guidance=(
        "Do the following steps one by one:\n"
        "1. Navigate to https://hkgx.com.hk/en\n"
        "2. Hover on the 'Market & Data' button to show more button options\n"
        "3. Click the 'History Price' button to access the historical price page\n"
        "4. Since the current date was selected, only need to select the option of RMB-denominated kilo gold\n"
        "5. Click the search button and have a look at the recent gold price trends\n"
        "6. Close the browser and give out a summary of recent gold price trends\n"
    ),
)

print("Final Result:\n\n")
print(result)

# Close the connection when done
playwright_connection.close()
print(f"\n✓ Connection closed: {not playwright_connection.is_connected}")

✓ Using temporary directory: /private/var/folders/9t/5r9fms9s5q33p6xty_0_k1mw0000gn/T/tmpspv3qdhb
✓ Connected to Playwright MCP server: connection-playwright-stdio
  Connection status: True
✓ Found 22 available browser tools

[ReCentAutoma]-[ReCentAutoma-7708d76f] is started.
[ReCentAutoma]-[ReCentAutoma-7708d76f] [__dynamic_step__] driving [initialize_task_goal]
[ReCentAutoma]-[ReCentAutoma-7708d76f] [__automa__] triggers [initialize_task_goal]
[ReCentAutoma]-[ReCentAutoma-7708d76f] 🎯 Task Goal
Find the recent gold prices on Hong Kong Gold Exchange website.

Do the following steps one by one:
1. Navigate to https://hkgx.com.hk/en
2. Hover on the 'Market & Data' button to show more button options
3. Click the 'History Price' button to access the historical price page
4. Since the current date was selected, only need to select the option of RMB-denominated kilo gold
5. Click the search button and have a look at the recent gold price trends
6. Close the browser and give out a summary of 

In this example, the agent autonomously uses a browser to navigate to the Hong Kong Gold Exchange website and search for the gold price. The entire execution process is fully recorded during the execution of `ReCentAutoma` (as shown above).

Because we launched the Playwright MCP Server with the `--save-video` option, a recording of the session is saved after execution. The entire browser operation can be viewed in the following video:

![type:video](../../../videos/HKGX-browser-demo.webm)

## Build a Multi-Agent System with ReCentAutoma

Bridgic’s component-based architecture makes it a natural choice for building Multi-Agent systems.

Because `ReCentAutoma` itself is an `Automa` (and also a `Worker`), it can be flexibly nested and combined with other automas. This allows you to create hierarchical agentic systems, where each agent manages its own goals, tools, and memory independently. There are two common methods for building a Multi-Agent system:

- **Utilizing a `ReCentAutoma` instance as a nested worker**, allowing for flexible and customized dynamic routing
- **Integrating `ReCentAutoma` as a kind of tool**, enabling intelligent and context-aware parameter filling

More articles on this topic will be published in the near future.